<div style="direction:rtl">با سلام خدمت استاد گرامی این فایل صرفا برای نمایش کد ها و جواب آن ها تهیه شده است در این جا قابلیت اجرای کد ها نیست </div>
<div style="direction:rtl">این قدم اول یک پروژه است در مرحله اول ما باید یک مدل ساده و ابتدایی طراحی میکردیم سپس نتیجه های به دست آمده را با مدل های گذشته مقایسه کنیم اگر مدل ما ارزش سرمایه گذاری داشته باشد مدل را پیشرفت خواهیم داد</div>

 

## Problem Statement

The max flow problem can be formulated mathematically as a linear programming problem using the following model.  

### Sets

 $N$ = nodes in the network  
 $A$ = network arcs

### Parameters

 $s$ = source node  
 $t$ = sink node  
 $c_{ij}$ = arc flow capacity, $\forall (i,j) \in A$
 
### Variables
 $f_{i,j}$ = arc flow, $\forall (i,j) \in A$

### Objective

Maximize the flow into the sink nodes  
 $\max \sum_{\{i \mid (i,t) \in A\}} c_{i,t} f_{i,t}$

### Constraints

Enforce an upper limit on the flow across each arc  
 $f_{i,j} \leq c_{i,j}$, $\forall (i,j) \in A$

Enforce flow through each node  
 $\sum_{\{i \mid (i,j) \in A\}} f_{i,j} = \sum_{\{i \mid (j,i) \in A\}} f_{j,i}$, $\forall j \in N - \{s,t\}$
 
Flow lower bound  
 $f_{i,j} \geq 0$, $\forall (i,j) \in A$

## Pyomo Formulation



In [1]:
from pyomo.environ import *

model = AbstractModel()

In [2]:
# Nodes in the network
model.N = Set()
# Network arcs
model.A = Set(within=model.N*model.N)

In [3]:
# Source node
model.s = Param(within=model.N)
# Sink node
model.t = Param(within=model.N)
# Flow capacity limits
model.c = Param(model.A)

In [4]:
# The flow over each arc
model.f = Var(model.A, within=NonNegativeReals)

In [5]:
# Maximize the flow into the sink nodes
def total_rule(model):
    return sum(model.f[i,j] for (i, j) in model.A if j == value(model.t))
model.total = Objective(rule=total_rule, sense=maximize)

In [6]:
# Enforce an upper limit on the flow across each arc
def limit_rule(model, i, j):
    return model.f[i,j] <= model.c[i, j]
model.limit = Constraint(model.A, rule=limit_rule)

# Enforce flow through each node
def flow_rule(model, k):
    if k == value(model.s) or k == value(model.t):
        return Constraint.Skip
    inFlow  = sum(model.f[i,j] for (i,j) in model.A if j == k)
    outFlow = sum(model.f[i,j] for (i,j) in model.A if i == k)
    return inFlow == outFlow
model.flow = Constraint(model.N, rule=flow_rule)

In [7]:
!cat maxflow.py

from pyomo.environ import *

model = AbstractModel()

# Nodes in the network
model.N = Set()
# Network arcs
model.A = Set(within=model.N*model.N)

# Source node
model.s = Param(within=model.N)
# Sink node
model.t = Param(within=model.N)
# Flow capacity limits
model.c = Param(model.A)

# The flow over each arc
model.f = Var(model.A, within=NonNegativeReals)

# Maximize the flow into the sink nodes
def total_rule(model):
    return sum(model.f[i,j] for (i, j) in model.A if j == value(model.t))
model.total = Objective(rule=total_rule, sense=maximize)

# Enforce an upper limit on the flow across each arc
def limit_rule(model, i, j):
    return model.f[i,j] <= model.c[i, j]
model.limit = Constraint(model.A, rule=limit_rule)

# Enforce flow through each node
def flow_rule(model, k):
    if k == value(model.s) or k == value(model.t):
        return Constraint.Skip
    inFlow  = sum(model.f[i,j] for (i,j) in model.A if j == k)
    outFlow = sum(model.f[i,j] for (i,j) in model.A if i == k)
    

## Model Data

<div style="direction:rtl">یکی از قابلیت های مدل ما این است که داده بعد از تشکیل شدن طرح مدل به آن تزریق میشود و میتوان از شیوه مرسوم برای بقیه نرم افزار ها هم استفاده کرد  </div>

In [8]:
!cat maxflow.dat

set N := Zoo A B C D E Home;
set A := (Zoo,A) (Zoo,B) (A,C) (A,D) (B,A) (B,C) (C,D) (C,E) (D,E) (D,Home) (E,Home);

param s := Zoo;
param t := Home;
param: c :=
Zoo A 11
Zoo B 8
A C 5
A D 8
B A 4
B C 3
C D 2
C E 4
D E 5
D Home 8
E Home 6;




## Solution

 

In [9]:
!pyomo solve --solver=glpk maxflow.py maxflow.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.02] Applying solver
[    0.06] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 14.0
    Solver results file: results.json
[    0.06] Applying Pyomo postprocessing actions
[    0.06] Pyomo Finished


By default, the optimization results are stored in the file `results.yml`:

In [10]:
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 14.0
  Upper bound: 14.0
  Number of objectives: 1
  Number of constraints: 17
  Number of variables: 12
  Number of nonzeros: 30
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.00943398475647
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------------

## References

* A. Schrijver, (2002). "On the history of the transportation and maximum flow problems". Mathematical Programming 91 (3): 437–445. 